In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
explore = pd.read_csv("result/consolidate_file.csv", nrows=100000)

In [3]:
explore['created_date'].sort_values()

36879    2023-05-01 00:00:00.197000+00:00
26559    2023-05-01 00:02:00.182000+00:00
37291    2023-05-01 00:04:00.092000+00:00
93804    2023-05-01 00:04:00.096000+00:00
79049    2023-05-01 00:06:00.081000+00:00
                       ...               
3884     2023-05-28 15:26:00.206000+00:00
9260     2023-05-28 15:26:00.251000+00:00
6565     2023-05-28 15:28:00.203000+00:00
56320    2023-05-28 15:30:00.136000+00:00
31800    2023-05-28 15:32:00.440000+00:00
Name: created_date, Length: 100000, dtype: object